### Extract Transform Load Process

1. In this project, data from a car dealer stored in different file formats will be extracted, transformed and loaded into a MySQL database. Thereafter, basic analysis will be performed against the data in the MySQL database

#### Import libraries

In [74]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime
import wget
import urllib.request
import pymysql
from mysql.connector import Error
import mysql.connector as sql
from sqlalchemy import create_engine

#### Download and Unzip Files

In [76]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
!unzip datasource.zip -d dealership_data

#### Set Paths

In [77]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file

#### Extraction and Merging of data from different file formats

In [79]:
#### CSV extract function
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

#### JSON extract function
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

#### XML extract function
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price","fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, 
                                      "price":price,"fuel":fuel}, ignore_index=True)
    return dataframe

#### Extraction and Joining of data into one file
def extract():
    # create an empty data frame to hold extracted data
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

#### Transformation of data

1.  Round the price column to 2 decimal places


In [80]:
def transform(data):
    data['price'] = round(data.price,2)
    return data

#### Loading the transformed data into a MySQL database

In [93]:
def connect_to_db():
    conn = sql.connect(host="localhost",user="root",passwd="admin123!",db="car_dealership",charset='utf8mb4')
    con_cursor=conn.cursor()
    con_cursor.execute('drop database if exists car_dealership')
    con_cursor.execute('create database car_dealership')
    con_cursor.execute('use car_dealership')
    con_cursor.execute('drop table if exists used_car_prices')
    con_cursor.execute('''create table used_car_prices(car_model varchar(255) null,year_of_manufacture int null,
                            price double null,fuel varchar(255) null)
                            ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci ROW_FORMAT=COMPRESSED''')
    conn.close()

def load(password,data_to_load):
    connect_to_db()
    # Creating an engine to quickly push data to car_dealership database
    host="localhost"
    database="car_dealership"
    port=3306
    user="root"
    connection_string = "mysql+pymysql://%s:%s@%s:%s/%s" % (user, password, host, port, database)
    engine = create_engine(connection_string)
    data_to_load.to_sql(name='used_car_prices',con=engine,if_exists='append',index=False)

#### Logging

In [82]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

#### Running ETL job

In [92]:
log("ETL Job Started")
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
log("Transform phase Started")
transformed_data = transform(extracted_data)

In [133]:
# Preview data before loading 
transformed_data.head()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel


In [ ]:
log("Transform phase Ended")
log("Load phase Started")
load("admin123!",transformed_data)
log("Load phase Ended")

#### Basic analysis on data in MySQL database

In [ ]:
conn = sql.connect(host="localhost",user="root",passwd="admin123!",db="car_dealership",charset='utf8mb4')

##### Yearly average price in 2 decimal place

In [121]:
pd.read_sql_query("""select year_of_manufacture,round(avg(price),2) yearly_avg_price 
                  from used_car_prices group by 1 order by round(avg(price),2) desc""",conn)

,year_of_manufacture,yearly_avg_price
0,2017,20315.09
1,2010,14639.30
2,2018,13805.97
3,2015,12126.87
4,2016,12113.80
5,2012,11835.82
6,2014,9667.91
7,2013,8922.06
8,2011,5046.64
9,2005,4656.72


##### Top 5 most expensive car model on average

In [118]:
pd.read_sql_query("""select car_model,round(avg(price) ,2) average_price
                    from used_car_prices group by 1
                  order by round(avg(price) ,2) desc limit 5""",conn)

,car_model,average_price
0,land cruiser,52238.81
1,fortuner,28671.64
2,innova,19773.63
3,vitara brezza,13805.97
4,ciaz,11152.57


##### Average price of fuel

In [132]:
pd.read_sql_query("""select fuel,round(avg(price) ,2) average_price
                    from used_car_prices group by 1
                  order by round(avg(price) ,2) desc limit 5""",conn)

,fuel,average_price
0,Diesel,16826.70
1,Petrol,6760.91
2,CNG,4626.87


#### Closing connection to MySQL database

In [135]:
conn.close()